<a href="https://colab.research.google.com/github/Karuni001/Finance-Management-Application/blob/main/UY6758GH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Submitted by : Karuna Gupta**

**Intern ID: RF/A1/A2018**

**Project Id: UY6758GH**  

**Task: Develop a Personal Finance Management Application**

1. User Registration and Authentication :
  - Implement user registration with unique usernames and passwords.
   - Add login functionality to authenticate users.



In [ ]:
from google.colab import drive
import sqlite3
import hashlib

# Step 1: Mount Google Drive
drive.mount('/content/drive')

# Database setup
def create_users_table():
    conn = sqlite3.connect('/content/drive/My Drive/finance.db')
    cursor = conn.cursor()
    cursor.execute('''CREATE TABLE IF NOT EXISTS users
                      (user_id INTEGER PRIMARY KEY AUTOINCREMENT,
                       username TEXT UNIQUE NOT NULL,
                       password_hash TEXT NOT NULL,
                       created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP)''')
    conn.commit()
    conn.close()



# Example usage
#if __name__ == "__main__":
#    create_users_table()
#
#    while True:
#        action = input("Choose an action (register, login, quit): ").strip().lower()
#        if action == "register":
#            register_user()
#        elif action == "login":
#            if login_user():
#                print("Welcome to your personal finance manager!")
#
#        elif action == "quit":
#            break
#        else:
#            print("Invalid action. Please try again.")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Registration functionality
def register_user():
    username = input("Enter a username: ")
    password = input("Enter a password: ")  # Using input() instead of getpass()
    password_hash = hashlib.sha256(password.encode()).hexdigest()

    conn = sqlite3.connect('finance.db')
    cursor = conn.cursor()

    try:
        cursor.execute("INSERT INTO users (username, password_hash) VALUES (?, ?)", (username, password_hash))
        conn.commit()
        print("User registered successfully!")
    except sqlite3.IntegrityError:
        print("Username already exists!")
    finally:
        conn.close()


In [ ]:
# Login functionality
def login_user():
    username = input("Enter your username: ")
    password = input("Enter your password: ")  # Using input() instead of getpass()
    password_hash = hashlib.sha256(password.encode()).hexdigest()

    conn = sqlite3.connect('finance.db')
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM users WHERE username = ? AND password_hash = ?", (username, password_hash))
    user = cursor.fetchone()
    conn.close()

    if user:
        print("Login successful!")
        return True
    else:
        print("Invalid username or password!")
        return False

2. Income and Expense Tracking :
   - Allow users to add, update, and delete income and expense entries.
   - Categorize transactions (e.g., Food, Rent, Salary, etc.).


In [ ]:
#Adding a Transaction:
def add_transaction(user_id, amount, category, transaction_type):
    conn = sqlite3.connect('finance.db')
    cursor = conn.cursor()
    cursor.execute('''INSERT INTO transactions (user_id, amount, category, transaction_type)
                      VALUES (?, ?, ?, ?)''',
                   (user_id, amount, category, transaction_type))
    conn.commit()
    conn.close()
    print(f"{transaction_type.capitalize()} of {amount} in category '{category}' added successfully!")


In [ ]:
#Updating a Transaction:
def update_transaction(transaction_id, amount=None, category=None):
    conn = sqlite3.connect('finance.db')
    cursor = conn.cursor()

    if amount:
        cursor.execute('''UPDATE transactions SET amount = ? WHERE transaction_id = ?''', (amount, transaction_id))
    if category:
        cursor.execute('''UPDATE transactions SET category = ? WHERE transaction_id = ?''', (category, transaction_id))

    conn.commit()
    conn.close()
    print(f"Transaction {transaction_id} updated successfully!")


In [ ]:
#Deleting a Transaction:
def delete_transaction(transaction_id):
    conn = sqlite3.connect('finance.db')
    cursor = conn.cursor()
    cursor.execute('''DELETE FROM transactions WHERE transaction_id = ?''', (transaction_id,))
    conn.commit()
    conn.close()
    print(f"Transaction {transaction_id} deleted successfully!")


In [ ]:
#def get_user_id(username):
#    cursor = conn.cursor()
#    cursor.execute('SELECT user_id FROM users WHERE username = ?', (username,))
#    user_id = cursor.fetchone()
#    return user_id[0] if user_id else None


In [ ]:
import sqlite3 # Import the sqlite3 module to work with SQLite databases

# ... (rest of your code)

def get_user_id(username):
    conn = sqlite3.connect('finance.db') # Establish a connection to your database
    cursor = conn.cursor()
    cursor.execute('SELECT user_id FROM users WHERE username = ?', (username,))
    user_id = cursor.fetchone()
    conn.close() # Close the connection after use
    return user_id[0] if user_id else None

# ... (rest of your code)

3. Financial Reports:
   - Generate monthly and yearly financial reports.
   - Calculate total income, expenses, and savings.

In [ ]:
#Calculate Monthly Financial Report:
import datetime

def get_monthly_report(user_id, year, month):
    conn = sqlite3.connect('/content/drive/My Drive/finance.db')
    cursor = conn.cursor()

    start_date = f"{year}-{month:02d}-01"
    end_date = f"{year}-{month:02d}-{datetime.datetime(year, month, 1).replace(day=28).day + 2:02d}"

    cursor.execute('''SELECT SUM(amount) FROM transactions
                      WHERE user_id = ? AND transaction_type = 'income'
                      AND date BETWEEN ? AND ?''', (user_id, start_date, end_date))
    total_income = cursor.fetchone()[0] or 0

    cursor.execute('''SELECT SUM(amount) FROM transactions
                      WHERE user_id = ? AND transaction_type = 'expense'
                      AND date BETWEEN ? AND ?''', (user_id, start_date, end_date))
    total_expense = cursor.fetchone()[0] or 0

    conn.close()

    savings = total_income - total_expense
    return {"month": month, "year": year, "income": total_income, "expenses": total_expense, "savings": savings}


In [ ]:
#Calculate Yearly Financial Report:
def get_yearly_report(user_id, year):
    conn = sqlite3.connect('/content/drive/My Drive/finance.db')
    cursor = conn.cursor()

    start_date = f"{year}-01-01"
    end_date = f"{year}-12-31"

    cursor.execute('''SELECT SUM(amount) FROM transactions
                      WHERE user_id = ? AND transaction_type = 'income'
                      AND date BETWEEN ? AND ?''', (user_id, start_date, end_date))
    total_income = cursor.fetchone()[0] or 0

    cursor.execute('''SELECT SUM(amount) FROM transactions
                      WHERE user_id = ? AND transaction_type = 'expense'
                      AND date BETWEEN ? AND ?''', (user_id, start_date, end_date))
    total_expense = cursor.fetchone()[0] or 0

    conn.close()

    savings = total_income - total_expense
    return {"year": year, "income": total_income, "expenses": total_expense, "savings": savings}

4. Budgeting:
   - Enable users to set monthly budgets for different categories.
   - Notify users when they exceed their budget limits.


In [ ]:
#Create Budget Table:
def create_budget_table():
    conn = sqlite3.connect('/content/drive/My Drive/finance.db')
    cursor = conn.cursor()
    cursor.execute('''CREATE TABLE IF NOT EXISTS budgets
                      (budget_id INTEGER PRIMARY KEY AUTOINCREMENT,
                       user_id INTEGER,
                       category TEXT NOT NULL,
                       amount REAL NOT NULL,
                       month INTEGER NOT NULL,
                       year INTEGER NOT NULL,
                       FOREIGN KEY (user_id) REFERENCES users(user_id))''')
    conn.commit()
    conn.close()


In [ ]:
#Set a Budget:
def set_budget(user_id, category, amount, year, month):
    conn = sqlite3.connect('/content/drive/My Drive/finance.db')
    cursor = conn.cursor()

    # Check if a budget already exists for this category, year, and month
    cursor.execute('''SELECT * FROM budgets
                      WHERE user_id = ? AND category = ? AND year = ? AND month = ?''',
                   (user_id, category, year, month))
    budget = cursor.fetchone()

    if budget:
        # Update the existing budget
        cursor.execute('''UPDATE budgets
                          SET amount = ?
                          WHERE user_id = ? AND category = ? AND year = ? AND month = ?''',
                       (amount, user_id, category, year, month))
        print(f"Budget updated for {category} in {month}/{year}.")
    else:
        # Insert a new budget
        cursor.execute('''INSERT INTO budgets (user_id, category, amount, year, month)
                          VALUES (?, ?, ?, ?, ?)''',
                       (user_id, category, amount, year, month))
        print(f"Budget set for {category} in {month}/{year}.")

    conn.commit()
    conn.close()


In [ ]:
#Check Budget Status:
def check_budget(user_id, year, month):
    conn = sqlite3.connect('/content/drive/My Drive/finance.db')
    cursor = conn.cursor()

    cursor.execute('''SELECT category, amount FROM budgets
                      WHERE user_id = ? AND year = ? AND month = ?''',
                   (user_id, year, month))
    budgets = cursor.fetchall()

    for category, budget_amount in budgets:
        cursor.execute('''SELECT SUM(amount) FROM transactions
                          WHERE user_id = ? AND category = ? AND transaction_type = 'expense'
                          AND strftime('%Y', date) = ? AND strftime('%m', date) = ?''',
                       (user_id, category, str(year), f'{month:02d}'))
        spent_amount = cursor.fetchone()[0] or 0

        if spent_amount > budget_amount:
            print(f"Warning: You have exceeded your budget for {category}! "
                  f"Budget: {budget_amount}, Spent: {spent_amount}")
        else:
            print(f"Budget status for {category} - Budget: {budget_amount}, Spent: {spent_amount}")

    conn.close()


5. Data Persistence :
   - Store user data and transactions in a SQLite or any other database as per your preference.
   - Implement functions to back up and restore data.


In [ ]:
#Backup Data:
import shutil

def backup_database():
    try:
        # Source database file
        db_path = '/content/drive/My Drive/finance.db'

        # Destination for backup
        backup_path = '/content/drive/My Drive/finance_backup.db'

        # Copy the database file to create a backup
        shutil.copyfile(db_path, backup_path)

        print("Database backup successful!")
    except Exception as e:
        print(f"Error during backup: {e}")


In [ ]:
#Restore Data:
def restore_database():
    try:
        # Backup file path
        backup_path = '/content/drive/My Drive/finance_backup.db'

        # Destination where the database will be restored
        db_path = '/content/drive/My Drive/finance.db'

        # Copy the backup file to the database location to restore it
        shutil.copyfile(backup_path, db_path)

        print("Database restoration successful!")
    except Exception as e:
        print(f"Error during restoration: {e}")


6. Testing and Documentation :
   - Write unit tests for key functionalities.
   - Create a user manual with installation and usage instructions


In [ ]:
#Unit Testing
import unittest
import sqlite3
import os

class TestFinanceApp(unittest.TestCase):

    def setUp(self):
        # Create a temporary database for testing
        self.db_path = 'test_finance.db'
        self.conn = sqlite3.connect(self.db_path)
        self.create_tables()

    def tearDown(self):
        # Close connection and remove test database after each test
        self.conn.close()
        os.remove(self.db_path)

    def create_tables(self):
        cursor = self.conn.cursor()
        cursor.execute('''CREATE TABLE IF NOT EXISTS users
                          (user_id INTEGER PRIMARY KEY AUTOINCREMENT,
                           username TEXT UNIQUE NOT NULL,
                           password TEXT NOT NULL)''')
        cursor.execute('''CREATE TABLE IF NOT EXISTS transactions
                          (transaction_id INTEGER PRIMARY KEY AUTOINCREMENT,
                           user_id INTEGER,
                           amount REAL NOT NULL,
                           category TEXT NOT NULL,
                           transaction_type TEXT NOT NULL,
                           date TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                           FOREIGN KEY (user_id) REFERENCES users(user_id))''')
        cursor.execute('''CREATE TABLE IF NOT EXISTS budgets
                          (budget_id INTEGER PRIMARY KEY AUTOINCREMENT,
                           user_id INTEGER,
                           category TEXT NOT NULL,
                           amount REAL NOT NULL,
                           month INTEGER NOT NULL,
                           year INTEGER NOT NULL,
                           FOREIGN KEY (user_id) REFERENCES users(user_id))''')
        self.conn.commit()

    def test_register_user(self):
        # Test user registration
        username = 'testuser'
        password = 'testpass'
        cursor = self.conn.cursor()
        cursor.execute('INSERT INTO users (username, password) VALUES (?, ?)', (username, password))
        self.conn.commit()

        cursor.execute('SELECT * FROM users WHERE username = ?', (username,))
        user = cursor.fetchone()

        self.assertIsNotNone(user)
        self.assertEqual(user[1], username)

    def test_add_transaction(self):
        # Test adding a transaction
        username = 'testuser'
        password = 'testpass'
        cursor = self.conn.cursor()
        cursor.execute('INSERT INTO users (username, password) VALUES (?, ?)', (username, password))
        user_id = cursor.lastrowid

        amount = 100.0
        category = 'Food'
        transaction_type = 'expense'
        cursor.execute('''INSERT INTO transactions (user_id, amount, category, transaction_type)
                          VALUES (?, ?, ?, ?)''', (user_id, amount, category, transaction_type))
        self.conn.commit()

        cursor.execute('SELECT * FROM transactions WHERE user_id = ? AND category = ?', (user_id, category))
        transaction = cursor.fetchone()

        self.assertIsNotNone(transaction)
        self.assertEqual(transaction[2], amount)

    def test_set_budget(self):
        # Test setting a budget
        username = 'testuser'
        password = 'testpass'
        cursor = self.conn.cursor()
        cursor.execute('INSERT INTO users (username, password) VALUES (?, ?)', (username, password))
        user_id = cursor.lastrowid

        category = 'Food'
        amount = 200.0
        month = 8
        year = 2024
        cursor.execute('''INSERT INTO budgets (user_id, category, amount, month, year)
                          VALUES (?, ?, ?, ?, ?)''', (user_id, category, amount, month, year))
        self.conn.commit()

        cursor.execute('SELECT * FROM budgets WHERE user_id = ? AND category = ?', (user_id, category))
        budget = cursor.fetchone()

        self.assertIsNotNone(budget)
        self.assertEqual(budget[2], category)

if __name__ == '__main__':
    # Create a test suite
    suite = unittest.TestLoader().loadTestsFromTestCase(TestFinanceApp)

    # Run the tests and get a result object
    result = unittest.TextTestRunner().run(suite)

    # Check if all tests passed
    if result.wasSuccessful():
        print("All tests passed!")
    else:
        print("Some tests failed.")

...
----------------------------------------------------------------------
Ran 3 tests in 0.109s

OK


All tests passed!


In [ ]:
def main():
    create_users_table()
    create_transactions_table()
    create_budget_table()

    while True:
        action = input("Choose an action (register, login, backup, restore, quit): ").strip().lower()
        if action == "register":
            register_user()
        elif action == "login":
            if login_user():
                username = input("Enter your username again: ")
                user_id = get_user_id(username)
                if not user_id:
                    print("User not found. Please try again.")
                    continue

                while True:
                    trans_action = input("Choose an action (add, update, delete, report, budget, check_budget, quit): ").strip().lower()
                    if trans_action == "add":
                        amount = float(input("Enter amount: "))
                        category = input("Enter category (e.g., Food, Rent, Salary): ").strip().capitalize()
                        transaction_type = input("Enter type (income or expense): ").strip().lower()
                        add_transaction(user_id, amount, category, transaction_type)
                    elif trans_action == "update":
                        trans_id = int(input("Enter transaction ID to update: "))
                        amount = input("Enter new amount (leave blank to keep current): ")
                        amount = float(amount) if amount else None
                        category = input("Enter new category (leave blank to keep current): ").strip().capitalize()
                        update_transaction(trans_id, amount, category)
                    elif trans_action == "delete":
                        trans_id = int(input("Enter transaction ID to delete: "))
                        delete_transaction(trans_id)
                    elif trans_action == "report":
                        report_type = input("Choose report type (monthly, yearly): ").strip().lower()
                        if report_type == "monthly":
                            year = int(input("Enter year (YYYY): "))
                            month = int(input("Enter month (MM): "))
                            report = get_monthly_report(user_id, year, month)
                            print(f"--- Monthly Report for {month}/{year} ---")
                            print(f"Total Income: {report['income']}")
                            print(f"Total Expenses: {report['expenses']}")
                            print(f"Savings: {report['savings']}")
                        elif report_type == "yearly":
                            year = int(input("Enter year (YYYY): "))
                            report = get_yearly_report(user_id, year)
                            print(f"--- Yearly Report for {year} ---")
                            print(f"Total Income: {report['income']}")
                            print(f"Total Expenses: {report['expenses']}")
                            print(f"Savings: {report['savings']}")
                        else:
                            print("Invalid report type. Please try again.")
                    elif trans_action == "budget":
                        year = int(input("Enter year (YYYY): "))
                        month = int(input("Enter month (MM): "))
                        category = input("Enter category (e.g., Food, Rent): ").strip().capitalize()
                        amount = float(input("Enter budget amount: "))
                        set_budget(user_id, category, amount, year, month)
                    elif trans_action == "check_budget":
                        year = int(input("Enter year (YYYY): "))
                        month = int(input("Enter month (MM): "))
                        check_budget(user_id, year, month)
                    elif trans_action == "quit":
                        break
                    else:
                        print("Invalid action. Please try again.")
        elif action == "backup":
            backup_database()
        elif action == "restore":
            restore_database()
        elif action == "quit":
            break
        else:
            print("Invalid action. Please try again.")

if __name__ == "__main__":
    main()


Choose an action (register, login, backup, restore, quit): login
Enter your username: karuna
Enter your password: karuna@01
Login successful!
Enter your username again: karuna
Choose an action (add, update, delete, report, budget, check_budget, quit): add
Enter amount: 50000
Enter category (e.g., Food, Rent, Salary): salary
Enter type (income or expense): income
Income of 50000.0 in category 'Salary' added successfully!
Choose an action (add, update, delete, report, budget, check_budget, quit): check_budet
Invalid action. Please try again.
Choose an action (add, update, delete, report, budget, check_budget, quit): budget
Enter year (YYYY): 20244
Enter month (MM): 08
Enter category (e.g., Food, Rent): rent
Enter budget amount: 30000
Budget set for Rent in 8/20244.
Choose an action (add, update, delete, report, budget, check_budget, quit): check_budget
Enter year (YYYY): 20244
Enter month (MM): 08
Budget status for Rent - Budget: 30000.0, Spent: 0
Choose an action (add, update, delete, r